In [109]:
import sqlalchemy as sa
import pandas as pd
import numpy as np
from conct_db import enginex


**LEEMOS LOS DATOS DESDE LA BD**

In [110]:
datos = pd.read_sql("""SELECT * FROM VisitasMasuno WHERE Fecha>'2022-07-01'""", con=enginex)
df_time = pd.DataFrame(datos)
df_time.sample(3)

,CodigoDistribuidor,CodigoVendedor,Fecha,VisitId,CheckinStart,CheckinEnd,IsVisitValid,VisitDuration,Cliente,HasBuyed,AtWorkingHours,IsInRoute
316124,369117,24.0,2022-07-06,320722750.0,2022-07-06 13:28:45,2022-07-06 13:32:33,True,228.0,1410.0,False,True,True
261303,445625,6.0,2022-07-05,319213418.0,2022-07-05 09:03:37,2022-07-05 09:45:24,True,2506.0,12673.0,False,True,True
69074,445625,12.0,2022-02-03,223043035.0,2022-02-03 09:32:38,2022-02-03 09:38:26,True,348.0,4124.0,False,True,True


**PROCESAMOS LOS DATOS PARA ADECUARLOS AL TRABAJO QUE VAMOS A REALIZAR**

In [111]:
# from sqlalchemy.types import INT
df_time['CodigoVendedor']=df_time['CodigoVendedor'].astype(int)
df_time['CodigoVendedor']=df_time['CodigoVendedor'].astype(str)
df_time['CodigoDistribuidor']=df_time['CodigoDistribuidor'].astype(str)
df_time['CodigoVendedor'].sample

<bound method NDFrame.sample of 0         23
1         25
2          1
3         35
4          2
          ..
401400     5
401401     5
401402     5
401403     5
401404     5
Name: CodigoVendedor, Length: 401405, dtype: object>

*CREAR EL CODIGO DE VENDEDOR CONCATENADO*

In [112]:
df_time['Cod_concat']=df_time.CodigoDistribuidor.str.cat(df_time['CodigoVendedor'])
# df_time['Cod_concat']=df_time.CodigoDistribuidor.astype(str)+df_time.CodigoVendedor.astype(str)
df_time.dtypes

CodigoDistribuidor            object
CodigoVendedor                object
Fecha                 datetime64[ns]
VisitId                      float64
CheckinStart          datetime64[ns]
CheckinEnd            datetime64[ns]
IsVisitValid                  object
VisitDuration                float64
Cliente                      float64
HasBuyed                      object
AtWorkingHours                object
IsInRoute                     object
Cod_concat                    object
dtype: object

In [113]:
df_time[['CodigoDistribuidor', 'CodigoVendedor','Cod_concat']]=df_time[['CodigoDistribuidor', 'CodigoVendedor','Cod_concat']].astype(int)
df_time.sample(3)

,CodigoDistribuidor,CodigoVendedor,Fecha,VisitId,CheckinStart,CheckinEnd,IsVisitValid,VisitDuration,Cliente,HasBuyed,AtWorkingHours,IsInRoute,Cod_concat
28139,328660,4,2022-01-11,206960850.0,2022-01-11 09:18:19,2022-01-11 09:22:38,True,259.0,878.0,False,True,True,3286604
285270,445625,17,2022-05-16,281985610.0,2022-05-16 12:33:47,2022-05-16 12:44:13,True,626.0,339.0,False,True,True,44562517
117840,70425,4,2022-04-26,270090684.0,2022-04-26 12:20:04,2022-04-26 12:20:39,False,35.0,2028.0,False,True,True,704254


**COMENZAMOS CON EL ANALISIS PROPIAMENTE DICHO**

In [114]:
df_time.groupby(['Cod_concat','Fecha'])['VisitDuration'].mean()/60

Cod_concat  Fecha     
704251      2022-01-08     3.711702
            2022-01-10     4.875926
            2022-01-11     3.796599
            2022-01-12     3.948148
            2022-01-13     2.826075
                            ...    
44562520    2022-07-18     4.138889
            2022-07-19     1.326667
            2022-07-21     4.431250
            2022-07-22     1.083333
            2022-07-25    23.566667
Name: VisitDuration, Length: 11663, dtype: float64

In [115]:
df_promedioxdia= df_time.groupby(['Cod_concat', 'Fecha'])['VisitDuration'].aggregate(['min', 'mean', 'max'])

In [116]:
df_promedioxdia.sort_values('mean')
df_promedioxdia.sample(15)

,,min,mean,max
Cod_concat,Fecha,,,
36911723,2022-04-11,22.0,392.210526,1469.0
36911733,2022-07-20,19.0,306.914286,2216.0
36911737,2022-04-04,472.0,1666.500000,4456.0
3691174,2022-02-24,2.0,238.333333,1279.0
36911733,2022-01-26,3.0,210.808511,1370.0
36911717,2022-03-31,2.0,186.121951,1213.0
44562511,2022-07-11,3.0,278.571429,3573.0
36911725,2022-03-21,2.0,194.309524,763.0
4456254,2022-05-14,2.0,100.102041,967.0


In [117]:
# df_time['id']=df_time.index
# df_time['id']=df_time['id'].astype(str)
# df_time['id']=df_time['id']+'b'
# df_time.sample()